Voglio customizzare il prompt ReAct in base al caso d'uso del chatbot, vedi [link](https://towardsdatascience.com/using-langchain-react-agents-for-answering-multi-hop-questions-in-rag-systems-893208c1847e)

In [2]:
import os
import telepot
from telepot.loop import MessageLoop
import datetime
import time
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
import re
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
prompt_react = hub.pull("hwchase17/react")
print(prompt_react.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [5]:
type(prompt_react)

langchain_core.prompts.prompt.PromptTemplate

In [3]:
new_prompt_react="""
Use the context given to you to answer the question.
When you come across time based questions like Has there been increase in sales from April last year and August last year,
use today's date as {today_date} for fetching exact start and end dates. Then your first action should be to fetch the sales data with metadata time between April 1st to April 30th.
That will be your answer 1. Your next action will be to fetch the sales data with metadata time between August 1st to
August 31st, this will be your answer 2. Now analyse and understand each of answer 1 and answer 2, calculate the
percentange change, compare them and answer the question.

You have access to the following tools:

{tools}
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""